In [1]:
import MeCab
import pandas as pd
import numpy as np

mecab = MeCab.Tagger()

In [73]:
import re

def mecab_nouns(text):
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile('.*\t[A-Z]+')
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split('\t')) for token in mecab.parse(text).splitlines()[:-1]]    
        
    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        if re.match('N[A-Z]+', token[1]):
            nouns.append(token[0])
    return nouns

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]] 
    
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])
    
    return morphs

def mecab_morphs_to_string(text):
    morphs = ""
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]] 

    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs += token[0]
        morphs += ' '
    return morphs

def mecab_pos(text):
    pos = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    # re()보다 re.compile()로 해서 미리 컴파일해두면 속도가 더 빠름
    pattern = re.compile(".*\t[A-Z]+") # 토큰이름, 탭, 품사종류
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    # group(0) -> 생성된 객체에서 스트링만 뽑아줌
    pos = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
    
    return pos

## 1. review.sorted.uniq.refined.tsv.text 만들기
- cut -f2 .\review.sorted.uniq.refined.tsv > .\review.sorted.uniq.refined.tsv.text
- review.sorted.uniq.refined.tsv의 text부분을 review.sorted.uniq.refined.tsv.text에 저장

In [7]:
dataset = pd.read_csv("review.sorted.uniq.refined.tsv", delimiter='\t', header=None)
dataset[1].to_csv('review.sorted.uniq.refined.tsv.text', index=False, header=None, sep="\t")

## 2. review.sorted.uniq.refined.tsv.label 만들기
- cut -f1 .\review.sorted.uniq.refined.tsv > .\review.sorted.uniq.refined.tsv.label
- review.sorted.uniq.refined.tsv의 label부분을 review.sorted.uniq.refined.tsv.label에 저장

In [94]:
dataset = pd.read_csv("review.sorted.uniq.refined.tsv", delimiter='\t', header=None)
dataset[0].to_csv('review.sorted.uniq.refined.tsv.label', index=False, header=None, sep="\t")

## 3. review.sorted.uniq.refined.tsv.text.tok 만들기
- cat .\review.sorted.uniq.refined.tsv.text | mecab -0 wakati | python .\post_tokenize.py .\review.sorted.uniq.refined.tsv.text > .\review.sorted.uniq.refined.tsv.text.tok

In [90]:
dataset = pd.read_csv("review.sorted.uniq.refined.tsv.text", delimiter='\t', header=None)

dataset = dataset.apply(lambda x :x.replace(to_replace=r'\s', value=' ▁', regex=True))
dataset.head(7) # '▁'

,0
0,!
1,! ▁다녹아서 ▁왓어요 ▁.. ▁짜증
2,!!!!!!!!!!!!!
3,!!!!이게뭐야?진짜이게뭐야?리뷰보고구매했는데!나원참!
4,싼 ▁게 ▁비지떡.200ml라고 ▁판매하는데 ▁내가 ▁보기엔 ▁절대 ▁이거 ▁100...
5,축 ▁결혼이라는 ▁리본도 ▁없고 ▁화환 ▁하나에 ▁두사람 ▁이름만 ▁있는 ▁걸 ▁보...
6,가죽 ▁검정이라는 ▁상품명에서 ▁누구나 ▁알 ▁수 ▁있 ▁듯 ▁인조가죽 ▁느낌의 ▁...


In [91]:
dataset[0] = dataset[0].apply(mecab_morphs_to_string)
dataset.head(7)

,0
0,!
1,! ▁ 다 녹 아서 ▁ 왓 어요 ▁.. ▁ 짜증
2,!!!!!!!!!!!!!
3,!!!! 이게 뭐 야 ? 진짜 이게 뭐 야 ? 리뷰 보고 구매 했 는데 ! 나원참 !
4,싼 ▁ 게 ▁ 비지떡 . 200 ml 라고 ▁ 판매 하 는데 ▁ 내 가 ▁ 보 기 ...
5,축 ▁ 결혼 이 라는 ▁ 리본 도 ▁ 없 고 ▁ 화환 ▁ 하나 에 ▁ 두 사람 ▁ ...
6,가죽 ▁ 검정 이 라는 ▁ 상품명 에서 ▁ 누구 나 ▁ 알 ▁ 수 ▁ 있 ▁ 듯 ▁...


In [92]:
dataset[0] = dataset[0].apply(lambda x: x.replace('▁ ', '▁'))
dataset.head(7)

,0
0,!
1,! ▁다 녹 아서 ▁왓 어요 ▁.. ▁짜증
2,!!!!!!!!!!!!!
3,!!!! 이게 뭐 야 ? 진짜 이게 뭐 야 ? 리뷰 보고 구매 했 는데 ! 나원참 !
4,싼 ▁게 ▁비지떡 . 200 ml 라고 ▁판매 하 는데 ▁내 가 ▁보 기 엔 ▁절대...
5,축 ▁결혼 이 라는 ▁리본 도 ▁없 고 ▁화환 ▁하나 에 ▁두 사람 ▁이름 만 ▁있...
6,가죽 ▁검정 이 라는 ▁상품명 에서 ▁누구 나 ▁알 ▁수 ▁있 ▁듯 ▁인조 가죽 ▁...


In [93]:
dataset.to_csv('review.sorted.uniq.refined.tsv.text.tok', index=False, header=None, sep="\t")

## 4. subword의 learn_bpe 적용해서 model 파일로 저장하기

python .\subword-nmt\learn_bpe.py --input .\review.sorted.uniq.refined.tsv.text.tok --output model --symbols 30000

## 5. subword의 apply_bpe 적용해서 review.sorted.uniq.refined.tsv.text.tok.bpe 파일로 저장하기

python .\subword-nmt\apply_bpe.py --codes .\model < .\review.sorted.uniq.refined.tsv.text.tok > .\review.sorted.uniq.refined.tsv.text.tok.bpe

### review.sorted.uniq.refined.tsv.text.tok.bpe 파일에서....
- '▁'  ~> 처음 파일의 띄어쓰기만
- '▁▁' ~> 처음 파일의 띄어쓰기 + subword로 분할한것
- ' ' ~> mecab으로 형태소 분할한것

## 6. 최종 파일 만들기
### review.sorted.uniq.refined.tsv.label와 review.sorted.uniq.refined.tsv.text.tok.bpe 합하기
- paste review.sorted.uniq.refined.tsv.label review.sorted.uniq.refined.tsv.text.tok.bpe > review.sorted.uniq.refined.tok.bpe


In [103]:
dataset1 = pd.read_csv("review.sorted.uniq.refined.tsv.label", delimiter='\t', header=None)
dataset2 = pd.read_csv("review.sorted.uniq.refined.tsv.text.tok.bpe", delimiter='\t', header=None)

ret_df = pd.concat([dataset1,dataset2], axis=1)
ret_df.to_csv('review.sorted.uniq.refined.tok.bpe', index=False, header=None, sep="\t")